In [1]:
import os
import fnmatch
import shutil
import json
from glob import glob
import pandas as pd

In [2]:
# 
key_dict = {
    'anat': ['\*AX*','_T1w'],
    'dwi': ['/*_DTI_*','_dwi'],
    'func':['/*Resting*','_task-rest_bold']    
}

In [3]:
class BIDS_fitting:
    """
    nii로 bids specfication에 맞게 조정하는 class
    """

    """
    개선 필요점
    1. structure compatible 한지 확인 가능하게 하기, 
    1-1 ex) sub-0023_ses-01 은 anatomy에서 Axial이 없었음, DICOM 폴더도 없었음
    1-2 ex) sub-0023_ses-01 / sub-0061_ses-01 에서 이동이 안되어서 수동으로 작업함
    """
    def set_path(self, source, target, key_dict):
        """
        Set the path and necessary variables

        Args:
        - source (string): Input path of source MRI data 
        - target (string): Input path of target MRI data 
        - key_dict (dictionary) : related variable, key = nii_type / value = list [찾는단어, 수정단어]
        """
        self.source = os.path.abspath(source)
        self.target = os.path.abspath(target)
        self.key_dict = key_dict
    
    def sub_structure(self):
        result = {}        
        subs = os.listdir(self.source)
        for sub in subs:
            level_zero_dirs = []
            sub = os.path.join(self.source,sub)
            result[sub] ={}  
            for root, dirs, files in os.walk(sub):
                cur_depth = root[len(sub):].count(os.sep)
                if cur_depth == 0:
                    level_zero_dirs.extend(dirs)
                    break
            
            for dir_name in level_zero_dirs:
                sub_path = os.path.join(sub, dir_name)
                ses_structure = {}
                
                for root, dirs, files in os.walk(sub_path):
                    cur_depth = root[len(sub_path):].count(os.sep)
                    if cur_depth not in ses_structure:
                        ses_structure[cur_depth] = []
                    ses_structure[cur_depth].extend(dirs)

                    if 'DICOM' in dirs:
                        ses_structure['dicom']= root         
                
                if sub not in result:
                    result[sub]= {}

                result[sub][dir_name] = ses_structure
        self.source_structure = result
    
    def glob_files(self,path):
        result = {}
        for key in self.key_dict.keys():
            result[key] = glob(path+self.key_dict[key][0])
        return result

    def copy(self):
        result = {}
        for key in self.source_structure.keys():            
            sub = key.split(os.sep)[-1]
            result[sub] = {}
            for ses in self.source_structure[key].keys():           
                path = self.source_structure[key][ses]['dicom']                
                nii_glob = self.glob_files(path)        
                sub_ses = sub+'_'+ses
                print(sub_ses+'_start')
                result[sub][ses] = {}    
                for nii_type in nii_glob.keys():
                    result[sub][ses][nii_type]={}
                    dst_folder = os.path.join(self.target,sub,ses,nii_type)
                    result[sub][ses][nii_type]['dst']=dst_folder
                    result[sub][ses][nii_type]['src_dst'] = []
                    result[sub][ses][nii_type]['json_task_name'] = [0]
                    
                    if not os.path.exists(dst_folder):
                        os.makedirs(dst_folder)
                        result[sub][ses][nii_type]['exist'] = 1
                    for file in nii_glob[nii_type]:
                        if fnmatch.fnmatch(file,'*.json'):
                            file_name = sub_ses+self.key_dict[nii_type][1]+'.json'
                            dst_file = os.path.join(dst_folder,file_name)
                            if not os.path.exists(dst_file):
                                shutil.copy2(file,dst_file)
                                result[sub][ses][nii_type]['src_dst'].append([file,dst_file])
                                if nii_type == 'func':
                                    with open(dst_file,'r') as f:
                                        json_data = json.load(f)
                                        json_data['TaskName'] = 'rest'
                                        result[sub][ses][nii_type]['json_task_name'].append(1)
                                    with open(dst_file,'w',encoding='utf-8') as make_file:
                                        json.dump(json_data,make_file,indent='\t')
                                        result[sub][ses][nii_type]['json_task_name'].append(2)
                                        print('{} is modified'.format(file_name))
                            else: result[sub][ses][nii_type]['src_dst'].append([file,'already exist'])
                            
                        elif fnmatch.fnmatch(file,'*.nii'):
                            file_name = sub_ses+self.key_dict[nii_type][1]+'.nii'
                            dst_file = os.path.join(dst_folder,file_name)
                            if not os.path.exists(dst_file):
                                shutil.copy2(file,dst_file)
                                result[sub][ses][nii_type]['src_dst'].append([file,dst_file])
                            else: result[sub][ses][nii_type]['src_dst'].append([file,'already exist']) 
                        elif fnmatch.fnmatch(file,'*.bval'):
                            file_name = sub_ses+self.key_dict[nii_type][1]+'.bval'
                            dst_file = os.path.join(dst_folder,file_name)
                            if not os.path.exists(dst_file):
                                shutil.copy2(file,dst_file)
                                result[sub][ses][nii_type]['src_dst'].append([file,dst_file])
                            else: result[sub][ses][nii_type]['src_dst'].append([file,'already exist']) 
                        elif fnmatch.fnmatch(file,'*.bvec'):
                            file_name = sub_ses+self.key_dict[nii_type][1]+'.bvec'
                            dst_file = os.path.join(dst_folder,file_name)
                            if not os.path.exists(dst_file):
                                shutil.copy2(file,dst_file)
                                result[sub][ses][nii_type]['src_dst'].append([file,dst_file])
                            else: result[sub][ses][nii_type]['src_dst'].append([file,'already exist']) 
                        print('{} is moved'.format(file))
                    
        self.copy_result = result


    def summary(self):
        rows = []
        for sub in self.copy_result.keys():
            for ses in self.copy_result[sub].keys():
                row = {'index': sub+'_'+ses}
                for nii_type in self.copy_result[sub][ses].keys():
                    row[nii_type] = len(self.copy_result[sub][ses][nii_type]['src_dst'])
                rows.append(row)

        df = pd.DataFrame(rows)
        df.set_index('index',inplace=True)
        self.summary = df
                




In [5]:
source = os.path.abspath(r'f:\3_Data\3_MRI\0_sourcedata')
raw = os.path.abspath(r'f:\3_Data\3_MRI\1_rawdata')  
a = BIDS_fitting()
a.set_path(source,raw,key_dict)
a.sub_structure()
a.copy()


sub-0001_ses-01_start
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_3D_AX_20210901084856_402.json is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_3D_AX_20210901084856_402.nii is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_DTI_32d_2mm_20210901084856_301.bval is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_DTI_32d_2mm_20210901084856_301.bvec is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_DTI_32d_2mm_20210901084856_301.json is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_DTI_32d_2mm_20210901084856_301.nii is moved
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_DTI_32d_2mm_20210901084856_301_ADC.nii is moved
sub-0001_ses-01_task-rest_bold.json is modified
f:\3_Data\3_MRI\0_sourcedata\sub-0001\ses-01\sub-0001_ses-01\sub-0001_ses-01_Resting_15

In [6]:
a.summary()
summary = a.summary
indices_with_zero = summary.index[summary.apply(lambda row: (row ==0).any(),axis=1)]
# Print the indices
print("Indices with at least one 0 in any column:")
for index in indices_with_zero:
    print(index)

Indices with at least one 0 in any column:
sub-0023_ses-01
sub-0061_ses-01


In [10]:
summary.to_csv(os.path.abspath(r'f:\3_Data\mri_summary.csv'))